# PCB Defect Detection with YOLOv12

## Business Objective

Electronics manufacturers face high scrap costs and yield losses due to rigid, legacy Automated Optical Inspection (AOI) systems that generate excessive false positives. This notebook demonstrates training a state-of-the-art **YOLOv12** object detection model directly inside Snowflake using GPU compute, enabling adaptive defect detection without moving sensitive image data off-platform.

## Technical Approach

We use the **Deep PCB** dataset containing 6 defect classes (open, short, mousebite, spur, copper, pin-hole) to train a YOLOv12 model on Snowflake's Container Runtime with GPU acceleration. The trained model is logged to **Snowflake's Model Registry** using the CustomModel API, enabling versioning, SQL inference, and SPCS deployment.

## Learning Objectives

After completing this notebook, you will understand:
1. How to use Snowflake Notebooks on Container Runtime with GPU compute pools
2. Converting industrial defect datasets to YOLO format
3. Training YOLOv12 for object detection tasks
4. Logging custom models to Snowflake's Model Registry using the CustomModel API

## Prerequisites

- **Mathematics**: Basic understanding of neural networks and loss functions
- **ML Concepts**: Familiarity with object detection, bounding boxes, IoU
- **Python**: Intermediate Python and PyTorch experience
- **Domain**: Basic PCB manufacturing knowledge helpful but not required

### Required Setup (Run Once)

Before executing this notebook, an administrator must configure external network access for GitHub:

```sql
-- Run sql/03_network_egress.sql as ACCOUNTADMIN, then:
ALTER NOTEBOOK CV_DEFECT_DETECTION_YOLO_NOTEBOOK
  SET EXTERNAL_ACCESS_INTEGRATIONS = (github_access_integration);
```

This enables the notebook to download the Deep PCB dataset from GitHub.

## Notebook Structure

| Section | Purpose |
|---------|----------|
| 1. Title & Objectives | Frame the problem |
| 2. Environment Setup | Install packages, imports, session |
| 3. Data Loading | Download from Snowflake stage |
| 4. Data Exploration | Visualize defect distribution |
| 5. Model/Algorithm | Convert to YOLO format, explain architecture |
| 6. Execution | Train YOLOv12 with diagnostics |
| 7. Evaluation | Loss curves, sample predictions |
| 8. Production Output | Log to Model Registry, write inference logs |
| 9. Key Takeaways | Summary and next steps |

## Output

- Trained YOLOv12 model logged to Snowflake Model Registry as `YOLO_PCB_DEFECT_DETECTOR`
- Model also saved to `@MODEL_STAGE/models/yolov12_pcb/yolo_best.pt` for Vision Lab
- Inference results written to `DEFECT_LOGS` table


---
## 2. Environment Setup

Install required packages and configure the runtime environment. We find the pip executable in the Container Runtime and install PyPI packages (ultralytics, supervision, opencv-python-headless).


In [ ]:
# =============================================================================
# PACKAGE INSTALLATION
# =============================================================================
# Install PyPI packages using pip (SPCS Container Runtime)

import os
import sys
import importlib

print("Installing packages...")
print(f"Python version: {sys.version}")

# Install packages using os.system (equivalent to !pip install in Jupyter)
packages = ["ultralytics", "supervision", "opencv-python-headless", "snowflake-ml-python", "torch"]

for pkg in packages:
    print(f"\n  Installing {pkg}...")
    exit_code = os.system(f"pip install {pkg} --quiet")
    if exit_code != 0:
        raise RuntimeError(f"Failed to install {pkg}: exit code {exit_code}")
    print(f"  [OK] {pkg} installed")

# Refresh import system
importlib.invalidate_caches()
print("\n[OK] Package installation complete")

# =============================================================================
# IMPORTS - All imports in same cell after install
# =============================================================================

# Standard library
import glob
import uuid
import shutil
from pathlib import Path
from datetime import datetime

# Data processing
import pandas as pd
import numpy as np
from PIL import Image
import yaml

# Visualization
import matplotlib.pyplot as plt

# ML/Computer Vision - import after pip install
from ultralytics import YOLO
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.distributors.pytorch import PyTorchDistributor, PyTorchScalingConfig, WorkerResourceConfig
from snowflake.ml.data.sharded_data_connector import ShardedDataConnector

# Snowflake
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col

print("[OK] All imports successful")


In [ ]:
# =============================================================================
# SNOWFLAKE SESSION SETUP
# =============================================================================
# Get active session from Container Runtime context

session = get_active_session()

# Verify context
print("Snowflake Session Info:")
print(f"  Database: {session.get_current_database()}")
print(f"  Schema: {session.get_current_schema()}")
print(f"  Warehouse: {session.get_current_warehouse()}")
print(f"  Role: {session.get_current_role()}")

# Helper function for fail-fast queries
def execute_query_helper(query: str, name: str = "query") -> pd.DataFrame:
    """
    Execute a SQL query with fail-fast error handling.
    
    Args:
        query: SQL query string
        name: Descriptive name for error messages
    
    Returns:
        DataFrame with query results
    
    Raises:
        RuntimeError: If query fails or returns None
    """
    try:
        result = session.sql(query).to_pandas()
        if result is None:
            raise RuntimeError(f"Query '{name}' returned None")
        return result
    except Exception as e:
        raise RuntimeError(f"Query '{name}' failed: {e}") from e

print("\n[OK] Session configured with fail-fast query helper")


In [ ]:
# =============================================================================
# VISUALIZATION SETUP - Dark Theme
# =============================================================================
# Configure matplotlib for Snowflake-inspired dark theme

plt.style.use('dark_background')
plt.rcParams.update({
    # Background colors (soft dark gray, not pure black)
    'figure.facecolor': '#121212',
    'axes.facecolor': '#121212',
    
    # Text colors (off-white to reduce glare)
    'text.color': '#E5E5E7',
    'axes.labelcolor': '#E5E5E7',
    'xtick.color': '#A1A1A6',
    'ytick.color': '#A1A1A6',
    
    # Grid and axes (subtle, not distracting)
    'axes.edgecolor': '#3A3A3C',
    'grid.color': '#2C2C2E',
    'grid.alpha': 0.6,
    
    # Figure quality and sizing
    'figure.dpi': 150,
    'savefig.dpi': 200,
    'figure.figsize': (12, 6),
    
    # Font configuration
    'font.family': 'sans-serif',
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
})

# Colorblind-safe palette for dark backgrounds
COLORS = ['#64D2FF', '#FF9F0A', '#5AC8FA', '#FFD60A', '#11567F', '#30D158']
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=COLORS)

print("[OK] Dark theme visualization configured")


---
## 3. Data Loading

Load the Deep PCB dataset using a hybrid approach:
1. **Fast path**: If data is cached in `@MODEL_STAGE/raw/deeppcb/`, download from stage
2. **First-time**: If stage is empty, clone directly from GitHub using sparse checkout

This approach eliminates the need for manual data upload during deployment.


In [ ]:
# =============================================================================
# HYBRID DATA LOADING: Stage Cache with Git Fallback
# =============================================================================
import subprocess

# Local paths in ephemeral container storage
RAW_DIR = "/tmp/raw_download"
DATASET_DIR = "/tmp/deeppcb_dataset"
IMG_DIR = os.path.join(DATASET_DIR, "images")
LBL_DIR = os.path.join(DATASET_DIR, "labels")

# Create directories
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(LBL_DIR, exist_ok=True)

# Check if data exists in stage (look for .jpg files)
print("Checking Snowflake stage for cached dataset...")
try:
    stage_files = session.sql(
        "LIST @MODEL_STAGE/raw/deeppcb/ PATTERN='.*\\.jpg'"
    ).collect()
except Exception:
    stage_files = []

if len(stage_files) > 10:
    # Stage has data - download from stage (preserving directory structure)
    print(f"[OK] Found {len(stage_files)} cached files in stage")
    print("Downloading from Snowflake stage (preserving structure)...")
    
    for row in stage_files:
        # row['name'] looks like: model_stage/raw/deeppcb/group00041/00041/00041001_temp.jpg
        stage_path = row['name']
        # Extract relative path after raw/deeppcb/
        if 'raw/deeppcb/' in stage_path:
            rel_path = stage_path.split('raw/deeppcb/')[-1]
            local_path = os.path.join(RAW_DIR, rel_path)
            
            # Create parent directories to preserve structure
            os.makedirs(os.path.dirname(local_path), exist_ok=True)
            
            # Download single file to its proper location
            session.file.get(f"@MODEL_STAGE/raw/deeppcb/{rel_path}", os.path.dirname(local_path))
    
    print(f"[OK] Downloaded {len(stage_files)} files to {RAW_DIR}")
else:
    # Stage empty - clone from GitHub (first-time setup)
    print("Stage empty. Cloning from GitHub (first-time setup)...")
    
    CLONE_DIR = "/tmp/DeepPCB"
    
    # Verify git is available
    git_check = subprocess.run(['which', 'git'], capture_output=True)
    if git_check.returncode != 0:
        raise RuntimeError(
            "git not found in container. "
            "Please upload data to @MODEL_STAGE/raw/deeppcb/ using deploy.sh --only-data"
        )
    
    # Clean previous clone attempt if exists
    if os.path.exists(CLONE_DIR):
        shutil.rmtree(CLONE_DIR)
    
    # Git sparse clone (only PCBData folder)
    print("  Running git sparse clone...")
    subprocess.run([
        'git', 'clone', '--filter=blob:none', '--sparse', '--depth=1',
        'https://github.com/tangsanli5201/DeepPCB.git', CLONE_DIR
    ], check=True)
    
    subprocess.run(
        ['git', 'sparse-checkout', 'set', 'PCBData'],
        cwd=CLONE_DIR, check=True
    )
    
    # Point RAW_DIR to cloned data
    RAW_DIR = os.path.join(CLONE_DIR, "PCBData")
    print(f"[OK] Cloned to {RAW_DIR}")
    
    # Cache to stage for future runs and Vision Lab access
    print("📤 Caching dataset to Snowflake stage for future runs...")
    upload_count = 0
    for group_dir in Path(RAW_DIR).glob("group*"):
        group_name = group_dir.name
        for sub_dir in group_dir.iterdir():
            if sub_dir.is_dir():
                sub_name = sub_dir.name
                for file_path in sub_dir.glob("*"):
                    if file_path.is_file():
                        stage_path = f"@MODEL_STAGE/raw/deeppcb/{group_name}/{sub_name}/"
                        session.file.put(str(file_path), stage_path, auto_compress=False, overwrite=True)
                        upload_count += 1
    print(f"[OK] Cached {upload_count} files to @MODEL_STAGE/raw/deeppcb/")

# List downloaded/cloned files
downloaded_files = list(Path(RAW_DIR).rglob("*"))
print(f"\n📁 Found {len(downloaded_files)} files/directories")


---
## 4. Data Exploration

Explore the Deep PCB dataset to understand the defect distribution and image characteristics before training.


In [ ]:
# =============================================================================
# EXPLORE DEEP PCB DATASET STRUCTURE
# =============================================================================

# Deep PCB class mapping (1-indexed in original dataset)
# Map to YOLO 0-indexed classes
CLASS_NAMES = {
    0: 'open',
    1: 'short', 
    2: 'mousebite',
    3: 'spur',
    4: 'copper',
    5: 'pin-hole'
}

# Original Deep PCB uses 1-6 indexing
DEEPPCB_TO_YOLO = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}

print("Deep PCB Defect Classes:")
print("="*50)
for idx, name in CLASS_NAMES.items():
    print(f"  Class {idx}: {name}")

# Find image and annotation files
image_files = list(Path(RAW_DIR).rglob("*.jpg")) + list(Path(RAW_DIR).rglob("*.png"))
txt_files = list(Path(RAW_DIR).rglob("*.txt"))

print(f"\n📁 Dataset Statistics:")
print(f"  Image files: {len(image_files)}")
print(f"  Annotation files: {len(txt_files)}")


In [ ]:
# =============================================================================
# VISUALIZE DEFECT CLASS DISTRIBUTION
# =============================================================================

# Count defects by class from annotation files
class_counts = {name: 0 for name in CLASS_NAMES.values()}

for txt_path in txt_files:
    try:
        with open(txt_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    # Deep PCB format: x1 y1 x2 y2 class_id
                    cls_id = int(parts[4])
                    if cls_id in DEEPPCB_TO_YOLO:
                        yolo_cls = DEEPPCB_TO_YOLO[cls_id]
                        class_counts[CLASS_NAMES[yolo_cls]] += 1
    except Exception:
        continue

# Create bar chart
fig, ax = plt.subplots(figsize=(10, 6))

classes = list(class_counts.keys())
counts = list(class_counts.values())

bars = ax.bar(classes, counts, color=COLORS[:len(classes)], edgecolor='white', linewidth=0.5)

# Add value labels on bars
for bar, count in zip(bars, counts):
    if count > 0:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                str(count), ha='center', va='bottom', fontsize=11, color='#E5E5E7')

ax.set_xlabel('Defect Class')
ax.set_ylabel('Count')
ax.set_title('Deep PCB Defect Distribution', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('/tmp/defect_distribution.png', dpi=150, bbox_inches='tight', facecolor='#121212')
plt.show()

print(f"\nTotal defects found: {sum(counts)}")


In [ ]:
# =============================================================================
# VISUALIZE SAMPLE PCB IMAGES WITH DEFECT ANNOTATIONS
# =============================================================================
# Display raw images with ground truth bounding boxes to understand what
# the model will learn to detect. This helps validate annotation quality
# and provides intuition for defect appearance.

import matplotlib.patches as patches

def draw_bbox_on_image(ax, img_path, txt_path, class_colors):
    """
    Draw an image with its ground truth bounding boxes overlaid.
    
    Args:
        ax: Matplotlib axis to draw on
        img_path: Path to the image file
        txt_path: Path to the annotation file (Deep PCB format)
        class_colors: Dict mapping class names to colors
    """
    # Load image
    img = Image.open(img_path)
    img_width, img_height = img.size
    ax.imshow(img, cmap='gray')
    
    # Parse annotations (Deep PCB format: x1 y1 x2 y2 class_id)
    if txt_path.exists():
        with open(txt_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    x1, y1, x2, y2, cls_id = map(int, parts[:5])
                    if cls_id in DEEPPCB_TO_YOLO:
                        yolo_cls = DEEPPCB_TO_YOLO[cls_id]
                        class_name = CLASS_NAMES[yolo_cls]
                        color = class_colors.get(class_name, '#FFFFFF')
                        
                        # Draw rectangle
                        rect = patches.Rectangle(
                            (x1, y1), x2-x1, y2-y1,
                            linewidth=2, edgecolor=color, facecolor='none'
                        )
                        ax.add_patch(rect)
                        
                        # Add label
                        ax.text(x1, y1-5, class_name, color=color, fontsize=8,
                               bbox=dict(boxstyle='round,pad=0.2', facecolor='#121212', alpha=0.7))

# Color mapping for defect classes
CLASS_COLORS = {
    'open': '#FF6B6B',      # Red-coral
    'short': '#4ECDC4',     # Teal
    'mousebite': '#FFE66D', # Yellow
    'spur': '#FF9F0A',      # Orange
    'copper': '#64D2FF',    # Blue
    'pin-hole': '#30D158'   # Green
}

# Find sample images with annotations
sample_pairs = []
for img_path in Path(RAW_DIR).rglob("*.jpg"):
    # Find corresponding annotation file
    img_name = img_path.stem
    base_id = img_name.replace('_temp', '').replace('_test', '')
    img_dir = img_path.parent
    parent_dir = img_dir.parent
    dir_name = img_dir.name
    not_dir = parent_dir / f"{dir_name}_not"
    txt_path = not_dir / f"{base_id}.txt"
    
    if txt_path.exists():
        sample_pairs.append((img_path, txt_path))
        if len(sample_pairs) >= 6:
            break

# Create visualization grid
if sample_pairs:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, (img_path, txt_path) in enumerate(sample_pairs):
        draw_bbox_on_image(axes[idx], img_path, txt_path, CLASS_COLORS)
        axes[idx].set_title(f'{img_path.name}', fontsize=10)
        axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(len(sample_pairs), 6):
        axes[idx].axis('off')
    
    # Add legend
    legend_elements = [patches.Patch(facecolor=color, edgecolor=color, label=name)
                       for name, color in CLASS_COLORS.items()]
    fig.legend(handles=legend_elements, loc='lower center', ncol=6, 
               fontsize=10, framealpha=0.8)
    
    plt.suptitle('Sample PCB Images with Ground Truth Annotations', 
                 fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('/tmp/sample_images_annotated.png', dpi=150, bbox_inches='tight', facecolor='#121212')
    plt.show()
    
    print(f"\nDisplayed {len(sample_pairs)} sample images with annotations")
else:
    print("[WARN] No image-annotation pairs found")


In [ ]:
# =============================================================================
# DATASET STATISTICS: Image Dimensions & Defects Per Image
# =============================================================================
# Understanding the dataset characteristics helps inform training parameters
# and identify potential issues (e.g., varying image sizes, class imbalance).

# Collect image dimensions and defect counts
image_dims = []
defects_per_image = []
defect_sizes = []

for img_path in Path(RAW_DIR).rglob("*.jpg"):
    try:
        # Get image dimensions
        with Image.open(img_path) as img:
            w, h = img.size
            image_dims.append((w, h))
        
        # Find annotation file
        img_name = img_path.stem
        base_id = img_name.replace('_temp', '').replace('_test', '')
        img_dir = img_path.parent
        parent_dir = img_dir.parent
        dir_name = img_dir.name
        not_dir = parent_dir / f"{dir_name}_not"
        txt_path = not_dir / f"{base_id}.txt"
        
        # Count defects and their sizes
        if txt_path.exists():
            defect_count = 0
            with open(txt_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        x1, y1, x2, y2 = map(int, parts[:4])
                        defect_count += 1
                        # Store defect size (area in pixels)
                        defect_sizes.append((x2 - x1) * (y2 - y1))
            defects_per_image.append(defect_count)
    except Exception:
        continue

# Create visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Image dimensions
if image_dims:
    widths = [d[0] for d in image_dims]
    heights = [d[1] for d in image_dims]
    axes[0].scatter(widths, heights, alpha=0.5, c=COLORS[0], s=20)
    axes[0].set_xlabel('Width (pixels)')
    axes[0].set_ylabel('Height (pixels)')
    axes[0].set_title('Image Dimensions Distribution', fontweight='bold')
    axes[0].grid(alpha=0.3)
    
    # Add annotation for common size
    from collections import Counter
    dim_counts = Counter(image_dims)
    most_common_dim, count = dim_counts.most_common(1)[0]
    axes[0].annotate(f'Most common: {most_common_dim[0]}x{most_common_dim[1]}\n({count} images)',
                     xy=most_common_dim, fontsize=9, color='#E5E5E7',
                     xytext=(10, -30), textcoords='offset points',
                     bbox=dict(boxstyle='round', facecolor='#2C2C2E', alpha=0.8))

# Plot 2: Defects per image
if defects_per_image:
    axes[1].hist(defects_per_image, bins=range(0, max(defects_per_image)+2), 
                 color=COLORS[1], edgecolor='white', linewidth=0.5, alpha=0.8)
    axes[1].set_xlabel('Number of Defects')
    axes[1].set_ylabel('Number of Images')
    axes[1].set_title('Defects Per Image Distribution', fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    # Add statistics
    avg_defects = np.mean(defects_per_image)
    axes[1].axvline(avg_defects, color='#FF6B6B', linestyle='--', linewidth=2, label=f'Mean: {avg_defects:.1f}')
    axes[1].legend()

# Plot 3: Defect sizes
if defect_sizes:
    axes[2].hist(defect_sizes, bins=30, color=COLORS[2], edgecolor='white', linewidth=0.5, alpha=0.8)
    axes[2].set_xlabel('Defect Area (pixels²)')
    axes[2].set_ylabel('Count')
    axes[2].set_title('Defect Size Distribution', fontweight='bold')
    axes[2].grid(axis='y', alpha=0.3)
    
    # Add median line
    median_size = np.median(defect_sizes)
    axes[2].axvline(median_size, color='#FF6B6B', linestyle='--', linewidth=2, label=f'Median: {median_size:.0f}px²')
    axes[2].legend()

plt.tight_layout()
plt.savefig('/tmp/dataset_statistics.png', dpi=150, bbox_inches='tight', facecolor='#121212')
plt.show()

# Print summary statistics
print("\nDataset Summary Statistics:")
print(f"  Total images analyzed: {len(image_dims)}")
print(f"  Images with annotations: {len(defects_per_image)}")
if defects_per_image:
    print(f"  Defects per image: min={min(defects_per_image)}, max={max(defects_per_image)}, avg={np.mean(defects_per_image):.1f}")
if defect_sizes:
    print(f"  Defect sizes (px²): min={min(defect_sizes)}, max={max(defect_sizes)}, median={np.median(defect_sizes):.0f}")


---
## 5. Model/Algorithm

Before implementing the model, we explain the data conversion process and YOLOv12 architecture.

### Deep PCB to YOLO Format Conversion

**What is the conversion?**

The Deep PCB dataset uses absolute pixel coordinates for bounding boxes, while YOLO requires normalized center-based coordinates.

**Deep PCB Format:** `x1 y1 x2 y2 class_id` (absolute pixels)

**YOLO Format:** `class_id x_center y_center width height` (normalized 0-1)

### Visual Representation

```
  Deep PCB Format (absolute px)          YOLO Format (normalized 0-1)
  ═══════════════════════════════        ═══════════════════════════════
                                         
  Image (W×H pixels)                     Image (1.0 × 1.0 normalized)
  ┌─────────────────────────┐            ┌─────────────────────────┐
  │                         │            │                         │
  │  (x1,y1)                │            │                         │
  │     ●─────────────┐     │            │         ●               │
  │     │             │     │    →→→     │      (cx,cy)            │
  │     │   DEFECT    │     │            │         w               │
  │     │             │     │            │       ◄───►             │
  │     └─────────────●     │            │         h               │
  │              (x2,y2)    │            │                         │
  │                         │            │                         │
  └─────────────────────────┘            └─────────────────────────┘
  
  Corner-based coordinates               Center-based, normalized
  (x1, y1) = top-left corner             cx = (x1+x2)/(2W)  ∈ [0,1]
  (x2, y2) = bottom-right corner         cy = (y1+y2)/(2H)  ∈ [0,1]
                                         w  = (x2-x1)/W     ∈ [0,1]
                                         h  = (y2-y1)/H     ∈ [0,1]
```

### Why Normalize?

1. **Scale Invariance**: Normalized coordinates work regardless of image resolution
2. **Consistent Range**: All values in [0,1] simplify neural network training
3. **Anchor-based Detection**: YOLO uses center-based anchors for prediction

### Mathematical Formulation

Given image dimensions $(W, H)$ and bounding box $(x_1, y_1, x_2, y_2)$:

$$x_{center} = \frac{x_1 + x_2}{2W}$$

$$y_{center} = \frac{y_1 + y_2}{2H}$$

$$width = \frac{x_2 - x_1}{W}$$

$$height = \frac{y_2 - y_1}{H}$$

All values are normalized to the range $[0, 1]$.


In [ ]:
# =============================================================================
# CONVERT DEEP PCB TO YOLO FORMAT
# =============================================================================

def convert_bbox_to_yolo(img_width: int, img_height: int, 
                         x1: int, y1: int, x2: int, y2: int) -> tuple:
    """
    Convert absolute pixel coordinates to YOLO normalized format.
    
    Args:
        img_width: Image width in pixels
        img_height: Image height in pixels
        x1, y1: Top-left corner coordinates
        x2, y2: Bottom-right corner coordinates
    
    Returns:
        Tuple of (x_center, y_center, width, height) normalized to [0,1]
        Each value is a float in range [0.0, 1.0]
    
    Example:
        >>> convert_bbox_to_yolo(640, 480, 100, 50, 200, 150)
        (0.234, 0.208, 0.156, 0.208)  # (cx, cy, w, h) normalized
    """
    # Calculate box dimensions (in pixels)
    box_width = x2 - x1    # int: width in pixels
    box_height = y2 - y1   # int: height in pixels
    
    # Calculate center point (in pixels)
    x_center = x1 + (box_width / 2.0)   # float: center x in pixels
    y_center = y1 + (box_height / 2.0)  # float: center y in pixels
    
    # Normalize to [0, 1] by dividing by image dimensions
    x_center_norm = x_center / img_width   # float ∈ [0, 1]
    y_center_norm = y_center / img_height  # float ∈ [0, 1]
    width_norm = box_width / img_width     # float ∈ [0, 1]
    height_norm = box_height / img_height  # float ∈ [0, 1]
    
    return x_center_norm, y_center_norm, width_norm, height_norm  # tuple(4,)


def process_deeppcb_dataset(raw_dir: str, img_out_dir: str, lbl_out_dir: str) -> int:
    """
    Process Deep PCB dataset and convert to YOLO format.
    
    Deep PCB structure:
      - Images in: group*/XXXXX/XXXXX001_temp.jpg (or _test.jpg)
      - Annotations in: group*/XXXXX_not/XXXXX001.txt
    
    Returns:
        Number of images processed
    """
    processed = 0
    
    # Find all image files
    image_files = list(Path(raw_dir).rglob("*.jpg"))
    print(f"  Found {len(image_files)} image files")
    
    for img_path in image_files:
        # Deep PCB naming: 00041001_temp.jpg -> annotation is 00041001.txt
        # Located in sibling _not directory
        img_name = img_path.stem  # e.g., "00041001_temp"
        
        # Remove _temp or _test suffix to get base ID
        base_id = img_name.replace('_temp', '').replace('_test', '')
        
        # Find annotation in _not directory (sibling to image directory)
        img_dir = img_path.parent  # e.g., .../group00041/00041/
        parent_dir = img_dir.parent  # e.g., .../group00041/
        dir_name = img_dir.name  # e.g., "00041"
        not_dir = parent_dir / f"{dir_name}_not"  # e.g., .../group00041/00041_not/
        
        txt_path = not_dir / f"{base_id}.txt"
        
        if not txt_path.exists():
            continue
        
        try:
            # Get image dimensions
            with Image.open(img_path) as img:
                img_width, img_height = img.size
            
            # Process annotations
            yolo_lines = []
            with open(txt_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                    
                    # Deep PCB: x1 y1 x2 y2 class_id
                    x1, y1, x2, y2, cls_id = map(int, parts[:5])
                    
                    if cls_id not in DEEPPCB_TO_YOLO:
                        continue
                    
                    yolo_cls = DEEPPCB_TO_YOLO[cls_id]
                    cx, cy, w, h = convert_bbox_to_yolo(
                        img_width, img_height, x1, y1, x2, y2
                    )
                    
                    yolo_lines.append(f"{yolo_cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")
            
            if yolo_lines:
                # Copy image to output directory
                out_img_path = Path(img_out_dir) / img_path.name
                if not out_img_path.exists():
                    shutil.copy2(img_path, out_img_path)
                
                # Write YOLO label file
                out_lbl_path = Path(lbl_out_dir) / (img_path.stem + '.txt')
                with open(out_lbl_path, 'w') as f:
                    f.write('\n'.join(yolo_lines))
                
                processed += 1
        
        except Exception as e:
            print(f"  [WARN] Error processing {img_path.name}: {e}")
            continue
    
    return processed


# Process the dataset
print("Converting Deep PCB to YOLO format...")
num_processed = process_deeppcb_dataset(RAW_DIR, IMG_DIR, LBL_DIR)
print(f"\n[OK] Processed {num_processed} images")
print(f"   Images saved to: {IMG_DIR}")
print(f"   Labels saved to: {LBL_DIR}")


In [ ]:
from pathlib import Path
# =============================================================================
# LOAD DATA TO SNOWFLAKE TABLE
# =============================================================================
# Load processed images and labels into a Snowflake table for distributed access
import base64

def load_dataset_to_table(image_dir, label_dir, table_name="PCB_LABELED_DATA"):
    print(f"Loading data from {image_dir} to table {table_name}...")
    
    records = []
    image_files = list(Path(image_dir).glob("*.jpg"))
    
    for img_path in image_files:
        # Read image as bytes
        with open(img_path, "rb") as f:
            img_bytes = f.read()
            img_b64 = base64.b64encode(img_bytes).decode("utf-8")
        
        # Read label
        lbl_path = Path(label_dir) / (img_path.stem + ".txt")
        label_text = ""
        if lbl_path.exists():
            with open(lbl_path, "r") as f:
                label_text = f.read()
        
        # Determine split (hash based for consistency)
        is_test = (hash(img_path.stem) % 10) < 2 # 20% test
        split = "TEST" if is_test else "TRAIN"
        
        records.append({
            "FILENAME": img_path.name,
            "IMAGE_BYTES": img_b64,
            "LABEL_TEXT": label_text,
            "SPLIT": split
        })
        
    if records:
        df = pd.DataFrame(records)
        session.write_pandas(
            df, 
            table_name, 
            auto_create_table=True, 
            overwrite=True
        )
        
        # =====================================================================
        # FAIL-FAST: Verify row count after write
        # =====================================================================
        verify_df = session.sql(f"SELECT COUNT(*) as CNT FROM {table_name}").to_pandas()
        if verify_df is None:
            raise RuntimeError(f"Verification query for {table_name} returned None")
        
        row_count = int(verify_df['CNT'].iloc[0])
        if row_count != len(records):
            raise RuntimeError(
                f"Output verification failed: wrote {len(records)} rows, "
                f"but table contains {row_count}"
            )
        
        print(f"[OK] Loaded and verified {row_count} rows in {table_name}")
        
        # Print split summary
        train_count = sum(1 for r in records if r['SPLIT'] == 'TRAIN')
        test_count = sum(1 for r in records if r['SPLIT'] == 'TEST')
        print(f"   Train: {train_count} images ({100*train_count/row_count:.1f}%)")
        print(f"   Test:  {test_count} images ({100*test_count/row_count:.1f}%)")
    else:
        raise RuntimeError(f"No images found in {image_dir}. Data processing step may have failed.")

load_dataset_to_table(IMG_DIR, LBL_DIR)


### What is YOLOv12?

**YOLOv12** (You Only Look Once, version 12) is the latest iteration in the YOLO family of real-time object detection models. Released in early 2025, it introduces significant architectural improvements over previous versions.

### Why YOLOv12 for PCB Defect Detection?

1. **Real-time Performance**: YOLO models process entire images in a single forward pass, enabling real-time inspection speeds required for production lines
2. **Small Object Detection**: PCB defects are often tiny (a few pixels); YOLOv12's improved feature pyramid handles multi-scale detection effectively
3. **Single-Stage Detection**: Unlike two-stage detectors (R-CNN family), YOLO directly predicts bounding boxes and class probabilities, reducing latency
4. **Transfer Learning**: Pre-trained weights from COCO dataset provide strong feature extractors that fine-tune well to industrial domains

### Architecture Overview

YOLOv12 builds on the CSPDarknet backbone with several key innovations:

```
Input Image (640x640)
        │
        ▼
┌───────────────────┐
│   CSPDarknet53    │  ← Backbone: Feature extraction with cross-stage connections
│   + Area Attention│  ← NEW: Efficient attention mechanism for global context
└───────────────────┘
        │
        ▼
┌───────────────────┐
│   PANet + SPPF    │  ← Neck: Multi-scale feature fusion
└───────────────────┘
        │
        ▼
┌───────────────────┐
│  Decoupled Head   │  ← Head: Separate classification and regression branches
└───────────────────┘
        │
        ▼
    [class, x, y, w, h, conf] × N detections
```

### Key Hyperparameters

| Parameter | Value | Purpose |
|-----------|-------|---------|
| `epochs` | 10 (demo) / 50+ (prod) | Training iterations over full dataset |
| `imgsz` | 640 | Input resolution; higher = more detail but slower |
| `batch` | 8 | Images per gradient update; limited by GPU memory |
| `conf` | 0.25 | Confidence threshold for inference |

### Loss Function

YOLOv12 uses a composite loss combining:

$$\mathcal{L}_{total} = \lambda_{box} \mathcal{L}_{box} + \lambda_{cls} \mathcal{L}_{cls} + \lambda_{dfl} \mathcal{L}_{dfl}$$

Where:
- $\mathcal{L}_{box}$: CIoU loss for bounding box regression
- $\mathcal{L}_{cls}$: Binary cross-entropy for class predictions  
- $\mathcal{L}_{dfl}$: Distribution focal loss for box refinement


---
## 6. Execution

Train the YOLOv12 model on the converted Deep PCB dataset.


In [ ]:
# =============================================================================
# CREATE YOLO DATA CONFIGURATION
# =============================================================================

data_config = {
    'path': DATASET_DIR,
    'train': 'images',  # Using same set for train/val in demo
    'val': 'images',
    'names': CLASS_NAMES
}

data_yaml_path = os.path.join(DATASET_DIR, 'data.yaml')
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"[OK] Created data.yaml at {data_yaml_path}")
print("\nConfiguration:")
print(yaml.dump(data_config, default_flow_style=False))

# =============================================================================
# DOWNLOAD PRETRAINED WEIGHTS FROM STAGE
# =============================================================================
# Download weights before distributed training (workers can't access GitHub)

WEIGHTS_DIR = "/tmp/weights"
os.makedirs(WEIGHTS_DIR, exist_ok=True)

print("\nDownloading pretrained YOLO weights from stage...")
session.file.get("@MODEL_STAGE/weights/yolo12n.pt", WEIGHTS_DIR)
WEIGHTS_PATH = os.path.join(WEIGHTS_DIR, "yolo12n.pt")

if os.path.exists(WEIGHTS_PATH):
    print(f"[OK] Weights downloaded to {WEIGHTS_PATH}")
else:
    raise RuntimeError("Failed to download pretrained weights from @MODEL_STAGE/weights/")


In [ ]:
# =============================================================================
# DIAGNOSTICS UTILITY
# =============================================================================
import time
class TrainingDiagnostics:
    """
    Simple diagnostics collector for distributed training.
    """
    def __init__(self):
        self.epoch_times = []
        self.start_time = None

    def start_epoch(self):
        self.start_time = time.time()

    def end_epoch(self, epoch, loss):
        duration = time.time() - self.start_time
        self.epoch_times.append(duration)
        print(f"Epoch {epoch}: Loss={loss:.4f}, Time={duration:.2f}s")


In [ ]:
# =============================================================================
# DISTRIBUTED TRAINING
# =============================================================================
def train_func_yolo():
    import os
    import yaml
    import shutil
    import base64
    import torch.distributed as dist
    from ultralytics import YOLO
    from snowflake.ml.modeling.distributors.pytorch import get_context
    
    # -------------------------------------------------------------------------
    # 1. Initialize distributed context FIRST (required for get_shard to work)
    # -------------------------------------------------------------------------
    context = get_context()
    rank = context.get_rank()
    dist.init_process_group(backend='nccl')
    
    # -------------------------------------------------------------------------
    # 2. Get data shard BEFORE clearing env vars (context detection needs them)
    # -------------------------------------------------------------------------
    dataset_map = context.get_dataset_map()
    shard = dataset_map['train'].get_shard()
    df = shard.to_pandas()
    
    # -------------------------------------------------------------------------
    # 3. NOW clear DDP env vars so YOLO uses single-GPU mode per worker
    # -------------------------------------------------------------------------
    for var in ['RANK', 'LOCAL_RANK', 'WORLD_SIZE', 'MASTER_ADDR', 'MASTER_PORT']:
        os.environ.pop(var, None)
    
    # -------------------------------------------------------------------------
    # 4. Setup directories and materialize data to disk for YOLO
    # -------------------------------------------------------------------------
    WORKER_ROOT = f"/tmp/worker_{rank}"
    IMG_DIR = f"{WORKER_ROOT}/images"
    LBL_DIR = f"{WORKER_ROOT}/labels"
    os.makedirs(IMG_DIR, exist_ok=True)
    os.makedirs(LBL_DIR, exist_ok=True)
    
    for idx, row in df.iterrows():
        with open(f"{IMG_DIR}/{row['FILENAME']}", "wb") as f:
            f.write(base64.b64decode(row['IMAGE_BYTES']))
        lbl_name = row['FILENAME'].rsplit('.', 1)[0] + ".txt"
        with open(f"{LBL_DIR}/{lbl_name}", "w") as f:
            f.write(row['LABEL_TEXT'])
            
    # -------------------------------------------------------------------------
    # 5. Create YOLO data config
    # -------------------------------------------------------------------------
    data_config = {
        'path': WORKER_ROOT,
        'train': 'images',
        'val': 'images',
        'names': {0: 'open', 1: 'short', 2: 'mousebite', 3: 'spur', 4: 'copper', 5: 'pin-hole'}
    }
    yaml_path = f"{WORKER_ROOT}/data.yaml"
    with open(yaml_path, 'w') as f:
        yaml.dump(data_config, f)
        
    # -------------------------------------------------------------------------
    # 6. Train YOLO model (using pre-downloaded weights from stage)
    # -------------------------------------------------------------------------
    model = YOLO("/tmp/weights/yolo12n.pt")
    results = model.train(
        data=yaml_path,
        epochs=10,
        imgsz=640,
        batch=16,
        project=f"{WORKER_ROOT}/runs",
        name="exp",
        device=0
    )
    
    # -------------------------------------------------------------------------
    # 7. Save model artifact using context.get_model_dir() for proper sync
    # -------------------------------------------------------------------------
    if rank == 0:
        src = f"{WORKER_ROOT}/runs/exp/weights/best.pt"
        if os.path.exists(src):
            # Save to context model dir (syncs to stage for multi-node)
            model_dir = context.get_model_dir()
            os.makedirs(model_dir, exist_ok=True)
            shutil.copy2(src, os.path.join(model_dir, "yolo_best.pt"))
            
            # Also copy to /tmp/models for backward compatibility
            os.makedirs("/tmp/models", exist_ok=True)
            shutil.copy2(src, "/tmp/models/yolo_best.pt")
            
    return results

# Launch
train_df = session.table("PCB_LABELED_DATA").filter(col("SPLIT") == "TRAIN")
distributor = PyTorchDistributor(
    train_func=train_func_yolo,
    scaling_config=PyTorchScalingConfig(
        num_nodes=1,
        num_workers_per_node=1,
        resource_requirements_per_worker=WorkerResourceConfig(num_cpus=0, num_gpus=1),
    )
)
distributor.run(dataset_map={"train": ShardedDataConnector.from_dataframe(train_df)})
print("Training Complete")


---
## 7. Evaluation

Analyze training results and visualize model performance.


In [ ]:
# =============================================================================
# VISUALIZE TRAINING RESULTS
# =============================================================================

# Define paths to training outputs (from distributed training worker 0)
PROJECT_DIR = "/tmp/worker_0/runs"
EXPERIMENT_NAME = "exp"

# Load training results CSV
results_csv = os.path.join(PROJECT_DIR, EXPERIMENT_NAME, 'results.csv')

if os.path.exists(results_csv):
    results_df = pd.read_csv(results_csv)
    results_df.columns = results_df.columns.str.strip()  # Clean column names
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Training Loss
    if 'train/box_loss' in results_df.columns:
        axes[0].plot(results_df['epoch'], results_df['train/box_loss'], 
                     label='Box Loss', color=COLORS[0], linewidth=2)
    if 'train/cls_loss' in results_df.columns:
        axes[0].plot(results_df['epoch'], results_df['train/cls_loss'], 
                     label='Class Loss', color=COLORS[1], linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training Loss Curves', fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Plot 2: mAP Metrics
    if 'metrics/mAP50(B)' in results_df.columns:
        axes[1].plot(results_df['epoch'], results_df['metrics/mAP50(B)'], 
                     label='mAP@50', color=COLORS[2], linewidth=2)
    if 'metrics/mAP50-95(B)' in results_df.columns:
        axes[1].plot(results_df['epoch'], results_df['metrics/mAP50-95(B)'], 
                     label='mAP@50-95', color=COLORS[3], linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('mAP')
    axes[1].set_title('Validation mAP Metrics', fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    axes[1].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig('/tmp/training_curves.png', dpi=150, bbox_inches='tight', facecolor='#121212')
    plt.show()
    
    # Print final metrics
    print("\nFinal Training Metrics:")
    final_row = results_df.iloc[-1]
    for col in results_df.columns:
        if 'mAP' in col or 'precision' in col or 'recall' in col:
            print(f"  {col}: {final_row[col]:.4f}")
    
    # =============================================================================
    # AUTOMATED CONVERGENCE INTERPRETATION
    # =============================================================================
    # Analyze training dynamics to provide actionable recommendations
    
    print("\n" + "="*50)
    print("CONVERGENCE ANALYSIS")
    print("="*50)
    
    # Get final mAP50 for interpretation
    final_map50 = final_row.get('metrics/mAP50(B)', 0)
    final_map50_95 = final_row.get('metrics/mAP50-95(B)', 0)
    
    # Analyze loss trends (is it still decreasing?)
    if 'train/box_loss' in results_df.columns and len(results_df) >= 3:
        recent_losses = results_df['train/box_loss'].tail(3).values
        loss_trend = recent_losses[-1] - recent_losses[0]
        loss_converging = loss_trend < 0
    else:
        loss_converging = True  # Assume converging if can't determine
    
    # Determine overall assessment
    if final_map50 >= 0.7:
        status = "[GOOD]"
        message = "Model has converged well with strong detection performance."
        recommendation = "Ready for evaluation on held-out test data."
    elif final_map50 >= 0.5:
        status = "[ACCEPTABLE]"
        message = "Model shows reasonable performance but may benefit from more training."
        if loss_converging:
            recommendation = "Consider increasing epochs to 30-50 for improved convergence."
        else:
            recommendation = "Loss may have plateaued. Try adjusting learning rate or augmentation."
    elif final_map50 >= 0.3:
        status = "[NEEDS IMPROVEMENT]"
        message = "Detection performance is below target. Model may be underfitting."
        recommendation = "Increase training epochs to 50+, or check data quality and augmentation."
    else:
        status = "[POOR]"
        message = "Model is not learning effectively."
        recommendation = "Check: 1) Data loading, 2) Label format, 3) Learning rate, 4) Batch size."
    
    print(f"\n  Status: {status}")
    print(f"  Assessment: {message}")
    print(f"  Recommendation: {recommendation}")
    
    # Additional insights
    print("\nPerformance Thresholds (for PCB defect detection):")
    print("  - Production-ready: mAP50 > 0.80")
    print("  - Acceptable: mAP50 > 0.60")
    print("  - Needs work: mAP50 < 0.50")
    print(f"\n  Current mAP@50: {final_map50:.3f}")
    print(f"  Current mAP@50-95: {final_map50_95:.3f}")
    
else:
    print("[WARN] Training results CSV not found - training may not have completed")


In [ ]:
# =============================================================================
# PER-CLASS PERFORMANCE BREAKDOWN
# =============================================================================
# Identify which defect classes are easier/harder to detect.
# This informs targeted data collection or augmentation strategies.

# Check if validation results are available
val_results_path = os.path.join(PROJECT_DIR, EXPERIMENT_NAME, 'results.csv')

if os.path.exists(val_results_path):
    # Load metrics CSV and look for per-class metrics
    metrics_df = pd.read_csv(val_results_path)
    metrics_df.columns = metrics_df.columns.str.strip()
    
    # YOLO stores per-class metrics in the final row or in separate files
    # Check for per-class columns or use the trained model to evaluate
    
    # If we have the trained model, we can get per-class metrics
    best_model_path = os.path.join(PROJECT_DIR, EXPERIMENT_NAME, 'weights', 'best.pt')
    
    if os.path.exists(best_model_path):
        print("Calculating per-class detection statistics...")
        
        eval_model = YOLO(best_model_path)
        
        # Run inference on all images and collect per-class stats
        class_stats = {name: {'tp': 0, 'fp': 0, 'fn': 0, 'conf_sum': 0, 'count': 0} 
                       for name in CLASS_NAMES.values()}
        
        test_images = list(Path(IMG_DIR).glob('*.jpg'))[:50]  # Sample for speed
        
        for img_path in test_images:
            results = eval_model.predict(str(img_path), conf=0.25, verbose=False)
            
            for box in results[0].boxes:
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                class_name = CLASS_NAMES.get(cls_id, 'unknown')
                
                if class_name in class_stats:
                    class_stats[class_name]['count'] += 1
                    class_stats[class_name]['conf_sum'] += conf
        
        # Calculate average confidence per class
        class_names_list = []
        detection_counts = []
        avg_confidences = []
        
        for name, stats in class_stats.items():
            class_names_list.append(name)
            detection_counts.append(stats['count'])
            avg_conf = stats['conf_sum'] / stats['count'] if stats['count'] > 0 else 0
            avg_confidences.append(avg_conf)
        
        # Create visualization
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: Detection counts by class
        bars1 = axes[0].bar(class_names_list, detection_counts, color=COLORS[:len(class_names_list)],
                           edgecolor='white', linewidth=0.5)
        axes[0].set_xlabel('Defect Class')
        axes[0].set_ylabel('Number of Detections')
        axes[0].set_title('Detections by Class (Sample of 50 Images)', fontweight='bold')
        axes[0].grid(axis='y', alpha=0.3)
        
        # Add value labels
        for bar, count in zip(bars1, detection_counts):
            if count > 0:
                axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                            str(count), ha='center', va='bottom', fontsize=10, color='#E5E5E7')
        
        # Plot 2: Average confidence by class
        bars2 = axes[1].bar(class_names_list, avg_confidences, color=COLORS[:len(class_names_list)],
                           edgecolor='white', linewidth=0.5)
        axes[1].set_xlabel('Defect Class')
        axes[1].set_ylabel('Average Confidence')
        axes[1].set_title('Average Detection Confidence by Class', fontweight='bold')
        axes[1].set_ylim(0, 1)
        axes[1].axhline(y=0.5, color='#FF6B6B', linestyle='--', alpha=0.7, label='Threshold')
        axes[1].grid(axis='y', alpha=0.3)
        axes[1].legend()
        
        # Add value labels
        for bar, conf in zip(bars2, avg_confidences):
            if conf > 0:
                axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                            f'{conf:.2f}', ha='center', va='bottom', fontsize=10, color='#E5E5E7')
        
        plt.tight_layout()
        plt.savefig('/tmp/perclass_performance.png', dpi=150, bbox_inches='tight', facecolor='#121212')
        plt.show()
        
        # Print insights
        print("\nPer-Class Analysis:")
        sorted_classes = sorted(zip(class_names_list, detection_counts, avg_confidences), 
                               key=lambda x: x[1], reverse=True)
        for name, count, conf in sorted_classes:
            status = "[GOOD]" if conf >= 0.6 else ("[WARN]" if conf >= 0.4 else "[POOR]")
            print(f"  {status} {name}: {count} detections, avg conf: {conf:.3f}")
        
        # Identify classes needing attention
        low_conf_classes = [name for name, _, conf in sorted_classes if conf < 0.5 and conf > 0]
        if low_conf_classes:
            print(f"\n[WARN] Classes with low confidence (< 0.5): {', '.join(low_conf_classes)}")
            print("   Consider: More training data or targeted augmentation for these classes.")
    else:
        print("[WARN] Best model not found for per-class evaluation")
else:
    print("[WARN] Validation results not available")


In [ ]:
# =============================================================================
# VISUALIZE SAMPLE PREDICTIONS
# =============================================================================

# Load best model
best_model_path = os.path.join(PROJECT_DIR, EXPERIMENT_NAME, 'weights', 'best.pt')

if os.path.exists(best_model_path):
    best_model = YOLO(best_model_path)
    
    # Get sample images for prediction
    sample_images = list(Path(IMG_DIR).glob('*.jpg'))[:4]
    
    if sample_images:
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        axes = axes.flatten()
        
        for idx, img_path in enumerate(sample_images):
            # Run inference - returns list of Results objects (one per image)
            results = best_model.predict(str(img_path), conf=0.25, verbose=False)
            # results[0]: Results object for first (only) image
            # results[0].boxes: Boxes object, shape (N, 6) for N detections
            # results[0].orig_shape: tuple (H, W) of original image
            
            # Plot with bounding boxes - returns numpy array (H, W, 3) BGR
            annotated = results[0].plot()
            
            # Convert BGR (OpenCV format) to RGB (matplotlib format)
            # annotated shape: (H, W, 3) -> reverse last axis for RGB
            annotated_rgb = annotated[:, :, ::-1]
            
            axes[idx].imshow(annotated_rgb)
            axes[idx].set_title(f'Sample {idx+1}: {img_path.name}', fontsize=10)
            axes[idx].axis('off')
            
            # Count detections
            num_detections = len(results[0].boxes)
            axes[idx].text(10, 30, f'Detections: {num_detections}', 
                          color='white', fontsize=10,
                          bbox=dict(boxstyle='round', facecolor='#121212', alpha=0.8))
        
        plt.suptitle('YOLOv12 PCB Defect Detection Results', fontsize=14, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.savefig('/tmp/sample_predictions.png', dpi=150, bbox_inches='tight', facecolor='#121212')
        plt.show()
    else:
        print("[WARN] No sample images found for prediction")
else:
    print(f"[WARN] Best model not found at {best_model_path}")


In [ ]:
# =============================================================================
# CONFIDENCE SCORE DISTRIBUTION
# =============================================================================
# Analyze the distribution of prediction confidences to understand model
# certainty and inform threshold selection for production deployment.

best_model_path = os.path.join(PROJECT_DIR, EXPERIMENT_NAME, 'weights', 'best.pt')

if os.path.exists(best_model_path):
    conf_model = YOLO(best_model_path)
    
    # Collect all confidence scores
    all_confidences = []
    confidence_by_class = {name: [] for name in CLASS_NAMES.values()}
    
    test_images = list(Path(IMG_DIR).glob('*.jpg'))
    
    for img_path in test_images:
        results = conf_model.predict(str(img_path), conf=0.1, verbose=False)  # Lower threshold to see distribution
        
        for box in results[0].boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            all_confidences.append(conf)
            
            class_name = CLASS_NAMES.get(cls_id, 'unknown')
            if class_name in confidence_by_class:
                confidence_by_class[class_name].append(conf)
    
    if all_confidences:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: Overall confidence distribution
        axes[0].hist(all_confidences, bins=50, color=COLORS[0], edgecolor='white', 
                    linewidth=0.5, alpha=0.8)
        
        # Add threshold markers
        thresholds = [(0.25, 'Default (0.25)', '#FF6B6B'),
                      (0.5, 'Medium (0.5)', '#FFD60A'),
                      (0.8, 'High (0.8)', '#30D158')]
        
        for thresh, label, color in thresholds:
            axes[0].axvline(x=thresh, color=color, linestyle='--', linewidth=2, label=label)
        
        axes[0].set_xlabel('Confidence Score')
        axes[0].set_ylabel('Number of Detections')
        axes[0].set_title('Overall Confidence Distribution', fontweight='bold')
        axes[0].legend(loc='upper right')
        axes[0].grid(axis='y', alpha=0.3)
        
        # Plot 2: Box plot by class
        box_data = [confidence_by_class[name] for name in CLASS_NAMES.values() if confidence_by_class[name]]
        box_labels = [name for name in CLASS_NAMES.values() if confidence_by_class[name]]
        
        if box_data:
            bp = axes[1].boxplot(box_data, labels=box_labels, patch_artist=True)
            
            # Color the boxes
            for patch, color in zip(bp['boxes'], COLORS[:len(box_data)]):
                patch.set_facecolor(color)
                patch.set_alpha(0.7)
            
            axes[1].axhline(y=0.5, color='#FF6B6B', linestyle='--', alpha=0.7, label='Threshold')
            axes[1].set_xlabel('Defect Class')
            axes[1].set_ylabel('Confidence Score')
            axes[1].set_title('Confidence Distribution by Class', fontweight='bold')
            axes[1].set_ylim(0, 1)
            axes[1].grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('/tmp/confidence_distribution.png', dpi=150, bbox_inches='tight', facecolor='#121212')
        plt.show()
        
        # Print statistics
        print("\nConfidence Score Statistics:")
        print(f"  Total detections (conf > 0.1): {len(all_confidences)}")
        print(f"  Mean confidence: {np.mean(all_confidences):.3f}")
        print(f"  Median confidence: {np.median(all_confidences):.3f}")
        print(f"  Std deviation: {np.std(all_confidences):.3f}")
        
        # Threshold analysis
        print("\nDetections at Different Thresholds:")
        for thresh in [0.25, 0.5, 0.75, 0.9]:
            count = sum(1 for c in all_confidences if c >= thresh)
            pct = 100 * count / len(all_confidences)
            print(f"  conf >= {thresh}: {count} detections ({pct:.1f}%)")
        
        # Recommendation
        high_conf_pct = sum(1 for c in all_confidences if c >= 0.7) / len(all_confidences) * 100
        if high_conf_pct > 60:
            print("\nRecommendation: Model shows high confidence. Safe to use threshold 0.5-0.7.")
        elif high_conf_pct > 30:
            print("\n[WARN] Recommendation: Mixed confidence. Consider threshold 0.3-0.5 with manual review.")
        else:
            print("\n[WARN] Recommendation: Low overall confidence. Review model or increase training epochs.")
    else:
        print("[WARN] No detections found for confidence analysis")
else:
    print("[WARN] Best model not found for confidence analysis")


---
## 8. Production Output

Log the trained model to Snowflake's Model Registry and write inference results to the DEFECT_LOGS table for dashboard visualization.

**Model Registry Benefits:**
- Version control for model iterations
- SQL-based inference via `MODEL!predict()` syntax
- Deployment to Snowpark Container Services for GPU inference
- Built-in model metadata and lineage tracking

**Output:**
- Model logged to registry as `YOLO_PCB_DEFECT_DETECTOR` version `v1`
- Model also saved to `@MODEL_STAGE/models/yolov12_pcb/yolo_best.pt` for Vision Lab compatibility
- Inference results written to `DEFECT_LOGS` table (enables Streamlit dashboard)


In [ ]:
# =============================================================================
# DEFINE CUSTOM MODEL CLASS FOR MODEL REGISTRY
# =============================================================================
# Wrap the YOLO model in a CustomModel class to enable logging to Snowflake's
# Model Registry. This allows versioning, SQL-based inference, and SPCS deployment.

from snowflake.ml.model import custom_model
from snowflake.ml.model import model_signature
import base64
import io
import numpy as np

# Class name mapping (must match training)
CLASS_NAMES_MAP = {
    0: 'open',
    1: 'short', 
    2: 'mousebite',
    3: 'spur',
    4: 'copper',
    5: 'pin-hole'
}

class YOLOPCBModel(custom_model.CustomModel):
    """
    Custom model wrapper for YOLOv12 PCB defect detection.
    
    This wrapper enables logging the YOLO model to Snowflake's Model Registry,
    which provides versioning, SQL inference, and SPCS deployment capabilities.
    
    Input: DataFrame with IMAGE_BYTES column (base64-encoded image data)
    Output: DataFrame with detection results (class, confidence, bbox coordinates)
    """
    
    def __init__(self, context: custom_model.ModelContext) -> None:
        super().__init__(context)
        from ultralytics import YOLO
        # Load YOLO model from the file provided in context
        self.model = YOLO(self.context["model_file"])
    
    @custom_model.inference_api
    def predict(self, input_df: pd.DataFrame) -> pd.DataFrame:
        """
        Run inference on base64-encoded images.
        
        Args:
            input_df: DataFrame with columns:
                - IMAGE_BYTES: base64-encoded image data (string)
                - FILENAME: optional, image filename for tracking
        
        Returns:
            DataFrame with columns:
                - FILENAME: source image filename
                - DETECTED_CLASS: defect class name
                - CONFIDENCE: detection confidence score
                - BBOX_X_CENTER: normalized bbox center x
                - BBOX_Y_CENTER: normalized bbox center y
                - BBOX_WIDTH: normalized bbox width
                - BBOX_HEIGHT: normalized bbox height
        """
        from PIL import Image
        
        results_list = []
        
        for idx, row in input_df.iterrows():
            # Decode base64 image
            img_bytes = base64.b64decode(row['IMAGE_BYTES'])
            img = Image.open(io.BytesIO(img_bytes))
            
            # Get filename if available
            filename = row.get('FILENAME', f'image_{idx}')
            
            # Run YOLO inference
            detections = self.model.predict(img, conf=0.25, verbose=False)
            
            # Process each detection
            for box in detections[0].boxes:
                cls_id = int(box.cls[0])
                confidence = float(box.conf[0])
                bbox = box.xywhn[0].tolist()  # Normalized coordinates
                
                results_list.append({
                    'FILENAME': filename,
                    'DETECTED_CLASS': CLASS_NAMES_MAP.get(cls_id, 'unknown'),
                    'CONFIDENCE': confidence,
                    'BBOX_X_CENTER': bbox[0],
                    'BBOX_Y_CENTER': bbox[1],
                    'BBOX_WIDTH': bbox[2],
                    'BBOX_HEIGHT': bbox[3]
                })
        
        # Return empty DataFrame with correct schema if no detections
        if not results_list:
            return pd.DataFrame(columns=[
                'FILENAME', 'DETECTED_CLASS', 'CONFIDENCE',
                'BBOX_X_CENTER', 'BBOX_Y_CENTER', 'BBOX_WIDTH', 'BBOX_HEIGHT'
            ])
        
        return pd.DataFrame(results_list)

print("[OK] YOLOPCBModel class defined for Model Registry")

In [ ]:
# =============================================================================
# LOG MODEL TO SNOWFLAKE MODEL REGISTRY
# =============================================================================
# Log the trained YOLO model to Snowflake's Model Registry using CustomModel.
# This enables model versioning, SQL-based inference, and SPCS deployment.

from snowflake.ml.model._signatures import snowpark_handler
from snowflake.ml.model.model_signature import FeatureSpec, DataType, ModelSignature

model_path = "/tmp/models/yolo_best.pt"

if os.path.exists(model_path):
    # -------------------------------------------------------------------------
    # 1. Define Model Signature (input/output schema)
    # -------------------------------------------------------------------------
    # Input: DataFrame with IMAGE_BYTES (base64 string) and optional FILENAME
    input_features = [
        FeatureSpec(name="IMAGE_BYTES", dtype=DataType.STRING),
        FeatureSpec(name="FILENAME", dtype=DataType.STRING),
    ]
    
    # Output: DataFrame with detection results
    output_features = [
        FeatureSpec(name="FILENAME", dtype=DataType.STRING),
        FeatureSpec(name="DETECTED_CLASS", dtype=DataType.STRING),
        FeatureSpec(name="CONFIDENCE", dtype=DataType.DOUBLE),
        FeatureSpec(name="BBOX_X_CENTER", dtype=DataType.DOUBLE),
        FeatureSpec(name="BBOX_Y_CENTER", dtype=DataType.DOUBLE),
        FeatureSpec(name="BBOX_WIDTH", dtype=DataType.DOUBLE),
        FeatureSpec(name="BBOX_HEIGHT", dtype=DataType.DOUBLE),
    ]
    
    predict_signature = ModelSignature(inputs=input_features, outputs=output_features)
    print("[OK] Model signature defined")
    
    # -------------------------------------------------------------------------
    # 2. Create Model Context with the trained weights file
    # -------------------------------------------------------------------------
    model_context = custom_model.ModelContext(model_file=model_path)
    yolo_custom_model = YOLOPCBModel(model_context)
    print("[OK] Custom model instance created")
    
    # -------------------------------------------------------------------------
    # 3. Log to Model Registry
    # -------------------------------------------------------------------------
    reg = Registry(
        session=session,
        database_name=session.get_current_database(),
        schema_name=session.get_current_schema()
    )
    
    MODEL_NAME = "YOLO_PCB_DEFECT_DETECTOR"
    VERSION_NAME = "v1"
    
    # -------------------------------------------------------------------------
    # 3a. Clean up existing version for idempotent execution
    # -------------------------------------------------------------------------
    try:
        existing_model = reg.get_model(MODEL_NAME)
        versions = [v.version_name for v in existing_model.versions()]
        
        if VERSION_NAME in versions:
            if len(versions) == 1:
                # Only version - delete entire model
                reg.delete_model(MODEL_NAME)
                print(f"[DELETED] Existing model {MODEL_NAME} (was only version)")
            else:
                # Multiple versions - just delete this version
                existing_model.delete_version(VERSION_NAME)
                print(f"[DELETED] Existing version {VERSION_NAME}")
    except Exception:
        # Model doesn't exist yet - this is expected on first run
        print(f"[INFO] No existing model {MODEL_NAME} found (first run)")
    
    # -------------------------------------------------------------------------
    # 3b. Log the model with SPCS as target platform (GPU inference)
    # -------------------------------------------------------------------------
    mv = reg.log_model(
        yolo_custom_model,
        model_name=MODEL_NAME,
        version_name=VERSION_NAME,
        pip_requirements=["ultralytics>=8.0.0", "opencv-python-headless", "pillow"],
        target_platforms=["SNOWPARK_CONTAINER_SERVICES"],
        signatures={"predict": predict_signature},
        comment="YOLOv12 PCB defect detection model trained on Deep PCB dataset (6 classes)"
    )
    
    print(f"[OK] Model logged to registry: {mv.model_name} version {mv.version_name}")
    print(f"📦 Model methods: {mv.show_functions()}")
    
    # Also upload to stage for backward compatibility with Vision Lab
    stage_path = "@MODEL_STAGE/models/yolov12_pcb/"
    session.file.put(model_path, stage_path, auto_compress=False, overwrite=True)
    print(f"📁 Also uploaded to stage for Vision Lab: {stage_path}yolo_best.pt")
    
else:
    print("[WARN] Model file not found at /tmp/models/yolo_best.pt")


In [ ]:
# =============================================================================
# TEST MODEL REGISTRY INFERENCE
# =============================================================================
# Verify the logged model works by running inference through the registry.

# Get a few test images from the processed dataset
test_images = list(Path(IMG_DIR).glob('*.jpg'))[:3]

if test_images and 'mv' in dir():
    print("Testing Model Registry inference...")
    
    # Prepare test data in the expected format
    test_records = []
    for img_path in test_images:
        with open(img_path, "rb") as f:
            img_b64 = base64.b64encode(f.read()).decode("utf-8")
        test_records.append({
            "IMAGE_BYTES": img_b64,
            "FILENAME": img_path.name
        })
    
    test_df = pd.DataFrame(test_records)
    print(f"Test input shape: {test_df.shape}")
    
    # Run inference through the model registry
    # Note: This uses SPCS for inference (GPU-enabled)
    try:
        result_df = mv.run(test_df, function_name="predict")
        
        print(f"\n[OK] Model Registry inference successful!")
        print(f"Output shape: {result_df.shape}")
        print("\nSample detections:")
        print(result_df.head(10).to_string(index=False))
        
        # Summary by class
        if len(result_df) > 0:
            class_summary = result_df.groupby('DETECTED_CLASS').agg({
                'CONFIDENCE': ['count', 'mean']
            }).round(3)
            print("\nDetection summary by class:")
            print(class_summary)
    except Exception as e:
        print(f"[WARN] Registry inference test failed: {e}")
        print("   This may be expected if SPCS is not yet deployed.")
        print("   The model is logged and can be deployed for inference later.")
else:
    if 'mv' not in dir():
        print("[WARN] Model version (mv) not available - run the logging cell first")
    else:
        print("[WARN] No test images found")

In [ ]:
# =============================================================================
# WRITE INFERENCE RESULTS TO DEFECT_LOGS TABLE
# =============================================================================
# Run inference on all processed images and write detection results to
# the DEFECT_LOGS table for dashboard visualization.

# Load the trained model
model_path = "/tmp/models/yolo_best.pt"

if os.path.exists(model_path):
    inference_model = YOLO(model_path)
    
    # Get all processed images
    test_images = list(Path(IMG_DIR).glob('*.jpg'))
    print(f"Running inference on {len(test_images)} images...")
    
    # Collect inference results
    inference_records = []
    
    for img_path in test_images:
        # Run inference
        results = inference_model.predict(str(img_path), conf=0.25, verbose=False)
        
        # Generate a board_id from filename (e.g., "00041001_temp.jpg" -> "PCB_00041001")
        img_stem = img_path.stem.replace('_temp', '').replace('_test', '')
        board_id = f"PCB_{img_stem}"
        
        # Stage path for the image
        image_stage_path = f"@MODEL_STAGE/raw/deeppcb/{img_path.name}"
        
        # Process each detection
        for box in results[0].boxes:
            cls_id = int(box.cls[0])
            confidence = float(box.conf[0])
            
            # Get normalized bbox (already in YOLO format: x_center, y_center, w, h)
            bbox = box.xywhn[0].tolist()  # Normalized coordinates
            
            inference_records.append({
                'INFERENCE_ID': str(uuid.uuid4()),
                'BOARD_ID': board_id,
                'INFERENCE_TIMESTAMP': datetime.now(),
                'DETECTED_CLASS': CLASS_NAMES.get(cls_id, 'unknown'),
                'CONFIDENCE_SCORE': confidence,
                'BBOX_X_CENTER': bbox[0],
                'BBOX_Y_CENTER': bbox[1],
                'BBOX_WIDTH': bbox[2],
                'BBOX_HEIGHT': bbox[3],
                'IMAGE_PATH': image_stage_path,
                'MODEL_VERSION': 'yolov12n_pcb_v1'
            })
    
    if inference_records:
        # Write to DEFECT_LOGS table
        inference_df = pd.DataFrame(inference_records)
        session.write_pandas(
            inference_df,
            'DEFECT_LOGS',
            auto_create_table=False,
            overwrite=False  # Append to existing data
        )
        
        print(f"[OK] Wrote {len(inference_records)} detections to DEFECT_LOGS table")
        
        # Show summary
        summary = inference_df.groupby('DETECTED_CLASS').size()
        print("\nDetection Summary:")
        for cls, count in summary.items():
            print(f"   {cls}: {count}")
        
        # Show sample of what was written
        print("\nSample records written:")
        print(inference_df[['DETECTED_CLASS', 'CONFIDENCE_SCORE', 'BOARD_ID']].head(10).to_string(index=False))
    else:
        print("[WARN] No detections found above confidence threshold")
else:
    print("[WARN] Model not found - run training first")

In [ ]:
# =============================================================================
# MODEL REGISTRY DEPLOYMENT OPTIONS
# =============================================================================
# The model has been logged to Snowflake's Model Registry with target platform 
# set to SNOWPARK_CONTAINER_SERVICES for GPU inference.
#
# DEPLOYMENT OPTIONS:
#
# 1. SPCS Model Serving (Recommended for production):
#    Deploy the model to SPCS for GPU-accelerated inference:
#    ```python
#    mv.deploy(
#        deployment_name="yolo_pcb_service",
#        target_method="predict",
#        compute_pool="GPU_POOL",
#        num_gpus=1
#    )
#    ```
#
# 2. SQL Inference (via SPCS):
#    Once deployed, call from SQL:
#    ```sql
#    SELECT YOLO_PCB_DEFECT_DETECTOR!predict(IMAGE_BYTES, FILENAME) 
#    FROM my_images_table;
#    ```
#
# 3. Python Inference (via Registry):
#    Use mv.run() as demonstrated in the test cell above.
#
# MODEL LOCATIONS:
# - Model Registry: YOLO_PCB_DEFECT_DETECTOR (version v1)
# - Stage backup: @MODEL_STAGE/models/yolov12_pcb/yolo_best.pt

print("[OK] Model logged to Snowflake Model Registry")
print("📦 Registry model: YOLO_PCB_DEFECT_DETECTOR v1")
print("📁 Stage backup: @MODEL_STAGE/models/yolov12_pcb/yolo_best.pt")
print("\nNext steps:")
print("   1. Deploy to SPCS with mv.deploy() for production inference")
print("   2. View model in Snowsight: AI & ML > Models")


---
## 9. Key Takeaways & Interpretation Guide

### What the Model Learned

The YOLOv12 model was trained to detect 6 classes of PCB manufacturing defects:

1. **Open circuits**: Broken or missing conductive traces
2. **Short circuits**: Unintended connections between traces
3. **Mousebites**: Irregular edge erosion on copper features
4. **Spurs**: Unwanted copper protrusions from traces
5. **Copper defects**: Missing or excess copper areas
6. **Pin-holes**: Small voids in copper layers

The model learns to identify these defects by:
- Extracting hierarchical visual features through the CSPDarknet backbone
- Fusing multi-scale features via PANet for detecting both small and large defects
- Predicting bounding boxes and class probabilities in a single forward pass

### Interpretation Guidelines

| Metric | Value Range | Interpretation |
|--------|-------------|----------------|
| **Confidence Score** | 0.0 - 1.0 | Model certainty; >0.7 = high confidence, 0.4-0.7 = review recommended, <0.4 = likely false positive |
| **mAP@50** | 0.0 - 1.0 | Detection accuracy at 50% IoU threshold; >0.8 = production-ready, >0.6 = acceptable |
| **mAP@50-95** | 0.0 - 1.0 | Stricter accuracy across IoU thresholds; typically 0.3-0.5 lower than mAP@50 |
| **Box Loss** | Decreasing | Should decrease during training; plateau indicates convergence |
| **Class Loss** | Decreasing | Classification accuracy improvement; should stabilize |

### Limitations & Considerations

1. **Demo Training Duration**: This notebook uses 10 epochs for demonstration. Production models should train for 50-100+ epochs for optimal performance.

2. **Dataset Size**: Deep PCB is a relatively small dataset (~1,500 images). Performance may improve with larger, more diverse training data.

3. **Class Imbalance**: Some defect classes (e.g., pin-hole) may have fewer training examples, leading to lower detection accuracy for rare defects.

4. **Domain Shift**: Model trained on Deep PCB dataset may require fine-tuning when applied to PCBs with different designs, lighting conditions, or camera setups.

5. **Real-time Requirements**: For production line integration, ensure inference latency meets throughput requirements (~30-50ms per image on GPU).

### Mathematical Recap

**YOLOv12 Loss Function:**

$$\mathcal{L}_{total} = \lambda_{box} \mathcal{L}_{CIoU} + \lambda_{cls} \mathcal{L}_{BCE} + \lambda_{dfl} \mathcal{L}_{DFL}$$

Where:
- $\mathcal{L}_{CIoU}$ = Complete IoU loss for bounding box regression (considers overlap, center distance, and aspect ratio)
- $\mathcal{L}_{BCE}$ = Binary cross-entropy for class predictions
- $\mathcal{L}_{DFL}$ = Distribution focal loss for fine-grained box coordinate prediction

**Confidence Threshold Selection:**

$$\text{Precision} = \frac{TP}{TP + FP}, \quad \text{Recall} = \frac{TP}{TP + FN}$$

Higher threshold → Higher precision, lower recall (fewer false positives, may miss defects)  
Lower threshold → Higher recall, lower precision (catch more defects, more false alarms)

### Further Learning Resources

- [Ultralytics YOLOv12 Documentation](https://docs.ultralytics.com/models/yolo12/)
- [Deep PCB Dataset Paper](https://arxiv.org/abs/1902.06197)
- [Snowflake Model Registry Documentation](https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/overview)
- [SPCS GPU Inference Guide](https://docs.snowflake.com/en/developer-guide/snowpark-container-services/overview)

### Next Steps

1. **Increase Training Epochs**: Re-run with 50+ epochs for production-quality model
2. **Hyperparameter Tuning**: Experiment with learning rate, batch size, and augmentation
3. **Deploy to SPCS**: Use `mv.deploy()` for GPU-accelerated inference endpoint
4. **Integrate with AOI System**: Connect model predictions to manufacturing execution system
5. **Monitor & Retrain**: Track detection accuracy over time and retrain on new defect examples
